In [1]:
!pip install torchvision torch pytorch-grad-cam


ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam


In [2]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.3-py3-none-any.whl size=38657 sha256=2b90ca715964067cffb1295c46603328061d0f75a12beb24d952b0f8289b3974
  Stored in directory: /root/.cache/pip/wheels/2e/ce/70/fe64f851895eae830b3c63ec7fc464cfa7c81aeb7ad4f68063
Successfully built grad-cam


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM # this should now work
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Path to your dataset (replace with your actual path in Google Drive)
img_folder = '/content/drive/MyDrive/mpox/'
infected_folder = img_folder + 'Monkeypox/'
non_infected_folder = img_folder + 'Others/' #Fixed typo: changed 'non_infected' to 'non-infected'


In [6]:
import os
img_paths = []
labels = []

In [7]:
# Assuming you have two folders 'infected' and 'non_infected'
# Collect infected images
for img_name in os.listdir(infected_folder):
    img_paths.append(os.path.join(infected_folder, img_name))
    labels.append(1)  # Label 1 for infected

In [8]:
# Collect non-infected images
for img_name in os.listdir(non_infected_folder):
    img_paths.append(os.path.join(non_infected_folder, img_name))
    labels.append(0)  # Label 0 for non-infected

In [9]:
# Define the MedicalDataset class
class MedicalDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB') # Ensure the image is loaded as RGB
        if self.transform:
            image = self.transform(image)
        return image, label

In [10]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [11]:
# Create dataset and dataloader
dataset = MedicalDataset(img_paths, labels, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [12]:
# Load the pre-trained EfficientNetV2-S model
model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)

# Replace the classifier layer to fit your number of classes
# Assuming binary classification (infected vs non-infected)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)  # 2 classes for binary classification

# Move the model to the appropriate device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 150MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25  # Adjust based on your dataset

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader)}')


In [ ]:
!pip install grad-cam

import torch
import torchvision.models as models
from torchvision import transforms
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# Assuming 'model' is your loaded EfficientNetV2-S model

# Target the last InvertedResidual block within the 'features' module
target_layers = [model.features[-1]]  # This selects the last layer in features, which is usually the last block

# ... (rest of your Grad-CAM code)
cam = GradCAM(model=model, target_layers=target_layers) #Remove use_cuda argument!pip install grad-cam

import torch
import torchvision.models as models
from torchvision import transforms
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# Assuming 'model' is your loaded EfficientNetV2-S model

# Target the last InvertedResidual block within the 'features' module
target_layers = [model.features[-1]]  # This selects the last layer in features, which is usually the last block

# ... (rest of your Grad-CAM code)
cam = GradCAM(model=model, target_layers=target_layers) #Remove use_cuda argument

In [ ]:
# Test the model on a new image
image_path = '/content/drive/MyDrive/mpox/Monkeypox/IMG_20211108_120232 (Custom).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)



In [ ]:
# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)


In [ ]:
# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0  # Normalize pixel values to the range [0, 1]

# Ensure img_np is of type float32
img_np = img_np.astype(np.float32)  # Convert to float32

# Ensure grayscale_cam is a 2D array
grayscale_cam = grayscale_cam[0, :]  # Assuming grayscale_cam has shape (1, 224, 224)

# Generate CAM
# ... your existing code ...
cam_image = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)

In [ ]:
# Display the result
plt.imshow(cam_image)
plt.show()

In [ ]:
# 8. Display the original image and the Grad-CAM overlay
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np) # Changed image_np to img_np

plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

#Applying on diffrent data sample _ Infected

In [ ]:
# Test the model on a new image
image_path = '/content/drive/MyDrive/mpox/Monkeypox/IMG_20211108_120232 (Custom).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

In [ ]:

# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0

# Ensure grayscale_cam has the correct number of channels
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

In [ ]:
# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

# 8. Display the original image and the Grad-CAM overlay
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

#Applying on diffrent data sample _ Infected

In [ ]:
# Test the model on a new image
image_path = '/content/drive/MyDrive/mpox/Monkeypox/IMG_20211108_120232 (Custom).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0

# Ensure grayscale_cam has the correct number of channels
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

# 8. Display the original image and the Grad-CAM overlay
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

#ScoreCAM


In [ ]:
!pip install pytorch-gradcam

from pytorch_grad_cam import ScoreCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# ... your existing code for model loading, transform, and device ...

# Test the model on a new image
image_path = '/content/drive/MyDrive/mpox/Monkeypox/IMG_20211108_120232 (Custom).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

# Initialize Score-CAM
cam = ScoreCAM(model=model, target_layers=target_layers) # Replace model and target_layers

# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0

# Ensure grayscale_cam has the correct number of channels
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

# Display the original image and the Score-CAM overlay
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Score-CAM')
plt.imshow(cam_image)
plt.show()


In [ ]:
!pip install pytorch-gradcam

from pytorch_grad_cam import ScoreCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# ... your existing code for model loading, transform, and device ...

# Test the model on a new image
image_path = '/content/drive/MyDrive/mpox/Monkeypox/M01_01_00.jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

# Initialize Score-CAM
cam = ScoreCAM(model=model, target_layers=target_layers) # Replace model and target_layers

# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0

# Ensure grayscale_cam has the correct number of channels
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

# Display the original image and the Score-CAM overlay
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Score-CAM')
plt.imshow(cam_image)
plt.show()


#GradCAMPlusPlus

In [ ]:
!pip install pytorch-gradcam

from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# ... your existing code for model loading, transform, and device ...

# Test the model on a new image
image_path = '/content/drive/MyDrive/mpox/Monkeypox/M04_01_00.jpg'
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

# Initialize GradCAM++
cam = GradCAMPlusPlus(model=model, target_layers=target_layers)

# Generate CAM
targets = [ClassifierOutputTarget(1)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# ... (Resize, handle channels, and visualize as in the previous example) ...
# Display the original
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Gradcam++')
plt.imshow(cam_image)
plt.show()

#XGradCAM

#EigenCAM

#LayerCAM

#Full-Gradient /  FullGrad

#Thresholded CAM (Binary Mask)
You can apply a threshold to the heatmap after generating it, converting the values into a binary mask (0/1). This highlights only the most activated regions.

OcclusionSensitivity